<a href="https://colab.research.google.com/github/yuks0810-dev/pj_ml/blob/main/signate-competition-987.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
# データ取得のためgoogle driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error

import lightgbm as lgb

warnings.filterwarnings("ignore")
%matplotlib inline
sns.set()

## Load data

In [ ]:
DRIVE_PATH = "/content/drive/MyDrive/pj_機械学習/SIGNATE/第43回_Beginner限定コンペ】国勢調査からの収入予測"

train_df = pd.read_csv(f'{DRIVE_PATH}/train.csv')
test_df = pd.read_csv(f'{DRIVE_PATH}/test.csv')
sample_submission_df = pd.read_csv(f'{DRIVE_PATH}/sample_submit.csv')

In [ ]:
sample_submission_df

,3873,0
0,3625,0
1,3028,0
2,13814,0
3,15398,0
4,13244,0
...,...,...
5094,6726,0
5095,1501,0
5096,6391,0
5097,7104,0


In [ ]:
# check the distribution of the target variable
train_df['Y'].value_counts()

0    8852
1    3048
Name: Y, dtype: int64

In [ ]:
test_df

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country
0,3873,17,Local-gov,132652,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
1,3625,23,Private,132652,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,United-States
2,3028,19,Private,132652,11th,7,Never-married,Handlers-cleaners,Own-child,White,Female,United-States
3,13814,30,State-gov,132652,HS-grad,9,Never-married,Protective-serv,Unmarried,Black,Female,United-States
4,15398,60,Private,132652,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...
5095,6726,19,Private,132652,Some-college,10,Never-married,Exec-managerial,Unmarried,White,Female,United-States
5096,1501,23,Private,132652,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,United-States
5097,6391,36,Private,132652,9th,5,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
5098,7104,33,Private,132652,Prof-school,11,Never-married,Prof-specialty,Not-in-family,White,Male,United-States


## Feature Engineering

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

categorical_feature_cols = ["marital-status", "education", "occupation", "native-country", "sex", "race", "relationship", "workclass"]

def apply_categorical_encode(df, feature_cols):


    # OneHotEncoderのインスタンス化
    encoder = LabelEncoder()

    for col in feature_cols:
        df[col] = encoder.fit_transform(df[col])

    return df

In [ ]:
train_df = apply_categorical_encode(train_df, categorical_feature_cols)

In [ ]:
train_df.columns

Index(['index', 'age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'native-country', 'Y'],
      dtype='object')

## Modeling

In [ ]:
# 特徴量とするカラム名をリストで保持しておきます。
features = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'native-country'
  ]


In [ ]:
# 学習データdevと検証データvalに分割します。
X = train_df.drop('Y', axis=1)  # 特徴量: 'Y'列を除外
y = train_df['Y']               # ラベル: 'Y'列のみ
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# lightgbmの学習を行っていきます。
# 代表的なハイパーパラメータを説明します。
# 詳しくはDocumentationを見てください。
# https://lightgbm.readthedocs.io/en/latest/Parameters.html

params = {
    # boosting_type: アルゴリズムの種類です。"gbdt"にしとけばいいです。
    # objective: 目的関数です。今回は回帰ですので"rmse"を使用します。
    # metric: 評価関数です。今回は目的関数と同じ"rmse"です。
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",

    # num_leaves: 葉の数の最大値です。決定木のmax_depthと類似していますが、あちらは木の深さであることに気をつけてください。私は大体16~128を調べます。
    # min_data_in_leaf: 葉に属する最小のデータ数です。20から始めて、過学習気味なら増やす感じです。
    'min_data_in_leaf': 100,
    'max_depth': 17,
    "min_data_in_leaf": 15,

    # num_iterations: GBDTは複数の決定木を逐次的に組み合わせて学習を行う機械学習モデルです。
    #                 num_iterationsはイテレート数を指定しますが、適当な大きな値に設定して良いです。
    # early_stopping_round: validデータのmetricが何回改善しなかったら学習を止めるか設定します。
    # learning_rate: 学習率です。0.1~0.001あたりを使います。GBDTはNNと違って学習率を下げればほぼ確実に精度が上がります。
    'num_iterations': 1000,
    "early_stopping_round": 200,
    'learning_rate': 0.01690366122001318,

    # bagging_fraction: ひとつの木が扱うデータの割合です。bagging_fractionの割合だけランダムサンプルします。
    #                   複数の木を組み合わせた時に、それぞれの木を微妙に異なるデータで学習させることで、全体の多様性を向上させます。
    #                   私は0.7~0.9あたりを使います。小さい方が過学習しにくいです。
    # bagging_freq: 上記のbaggingをいくつ行うかです(多分)。私はいつも5にしてます。
    # feature_fraction: ひとつの木が扱う特徴量の割合です。feature_fractionの割合だけランダムサンプルします。
    #                   私は0.5~0.9あたりを使います。今は特徴量が少ししかないので1.0にしています。
    'bagging_fraction': 0.2,
    "bagging_freq": 1,
    "feature_fraction": 1.0,

    'verbose': -1,
    'num_leaves': 29
}

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from lightgbm import LGBMClassifier
# import scipy.stats as stats

# param_dist = {
#     "num_leaves": (100, 1000),
#     "min_data_in_leaf": (10, 100),
#     'num_leaves': stats.randint(10, 50),
#     'learning_rate': stats.uniform(0.01, 0.2),
#     'max_depth': stats.randint(5, 20),
#     "num_iterations": (1000, 12000),
#     "bagging_fraction": (0.2, 0.9),
#     "bagging_freq": (1, 10),
# }

# lgbm = LGBMClassifier()
# random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=100, cv=5)
# random_search.fit(X, y)

In [ ]:
# best_params = random_search.best_params_

# best_params

In [ ]:
# 'index'列を除外
X_train_no_index = X_train.drop('index', axis=1)
X_val_no_index = X_val.drop('index', axis=1)

# LightGBMのモデルインスタンスを作成
clf = lgb.LGBMClassifier(**params, num_boost_round=1000, early_stopping_rounds=200)

# モデルの学習
clf.fit(
    X_train_no_index, y_train,
    eval_set=[(X_val_no_index, y_val)],
    categorical_feature=categorical_feature_cols,
)


LGBMClassifier(bagging_fraction=0.2, bagging_freq=1, early_stopping_round=200,
               early_stopping_rounds=200, feature_fraction=1.0,
               learning_rate=0.01690366122001318, max_depth=17,
               metric='binary_logloss', min_data_in_leaf=15,
               num_boost_round=1000, num_iterations=1000, num_leaves=29,
               objective='binary', verbose=-1)

In [ ]:
# categorical encoding for test data

test_df = apply_categorical_encode(test_df, categorical_feature_cols)

In [ ]:
# 予測を行います。
dev_pred = clf.predict(X_train[features])
val_pred = clf.predict(X_val[features])

# 閾値を設定
threshold = 0.5

# 予測確率が閾値以上の場合は1, それ以下の場合は0に二値化
dev_pred = np.where(dev_pred >= threshold, 1, 0)
val_pred = np.where(val_pred >= threshold, 1, 0)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 予測値は既に計算されていると仮定（dev_pred, val_pred）
# dev_pred = clf.predict(X_train)
# val_pred = clf.predict(X_val)

# 実際のラベルと予測値を用いて精度を計算
accuracy_dev = accuracy_score(y_train, dev_pred)
accuracy_val = accuracy_score(y_val, val_pred)

print(f"Accuracy on development set: {accuracy_dev:.3f}")
print(f"Accuracy on validation set: {accuracy_val:.3f}")

# ROC-AUCスコアの計算には、予測確率が必要
# ここでは、予測確率を取得するためにpredict_probaを使用し、正のクラスの確率を使用します
dev_proba = clf.predict_proba(X_train.drop('index', axis=1))[:, 1]  # 予測確率を取得
val_proba = clf.predict_proba(X_val.drop('index', axis=1))[:, 1]

# ROC-AUCスコアを計算
roc_auc_dev = roc_auc_score(y_train, dev_proba)
roc_auc_val = roc_auc_score(y_val, val_proba)

print(f"ROC-AUC on development set: {roc_auc_dev:.3f}")
print(f"ROC-AUC on validation set: {roc_auc_val:.3f}")



Accuracy on development set: 0.857
Accuracy on validation set: 0.838
ROC-AUC on development set: 0.920
ROC-AUC on validation set: 0.892


In [ ]:
test_pred = clf.predict(test_df[features])

In [ ]:
import pandas as pd
import os
from datetime import datetime
import csv

submission = pd.DataFrame({
    'ID': test_df['index'],  # または適切なID列
    'Prediction': test_pred
})

# 現在の日付と時間を取得してファイル名を生成
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
new_file_name = f'modified_{current_time}.csv'

file_path = "final_submission.csv"

# CSVファイルとして保存
submission_csv = submission.to_csv(file_path, index=False)

# 元のCSVファイル名と新しいCSVファイル名
input_file = "final_submission.csv"
output_file = new_file_name

# CSVファイルを読み込む
with open(input_file, 'r', newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    data = list(csvreader)

# 1行目を削除
data.pop(0)

# 新しいCSVファイルにデータを書き込む
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print("1行目のデータを削除しました。", output_file)


os.remove(file_path)

print("処理の終了")

1行目のデータを削除しました。 modified_20240206_231436.csv
処理の終了
